# Imports

In [ ]:
import sys

import numpy as np
import torch

from captum.robust import PGD

sys.path.append('/mnt/home/rheinrich/taaowpf')

from data.cnn.wpf_dataset_germany_all_experiments import WPF_Germany_DataModule
from models.cnn.resnet import WPF_ResNet
from robustness_evaluation.noise_attack import NoiseAttack
from robustness_evaluation.robustness_evaluator import AdversarialRobustnessEvaluator
from robustness_evaluation.robustness_scores import MSELossBatch, MSELossBatch_SemiTargeted, RMSELossBatch, RMSELossBatch_Area

# Robustness evaluation for all experiments, semi-targets, and targets

In [ ]:
target_attacker_dict = {"decreasing": [0.70, 0.55, 0.45, 0.30, 0.20, 0.17, 0.15, 0.12],
                       "increasing": [0.25, 0.40, 0.50, 0.60, 0.65, 0.72, 0.78, 0.82], 
                       "zigzag": [0.15, 0.30, 0.15, 0.30, 0.15, 0.30, 0.15, 0.30],
                       "constant": [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],}

In [ ]:
semi_target_attacker_dict = {"low": {'lower_bound': torch.Tensor(np.repeat(0., 8)), 'upper_bound': torch.Tensor(np.repeat(0.25, 8)),},
                            "medium": {'lower_bound': torch.Tensor(np.repeat(0.25, 8)), 'upper_bound': torch.Tensor(np.repeat(0.5, 8)),},
                            "high": {'lower_bound': torch.Tensor(np.repeat(0.5, 8)), 'upper_bound': torch.Tensor(np.repeat(0.75, 8)),}, 
                            "very high": {'lower_bound': torch.Tensor(np.repeat(0.75, 8)), 'upper_bound': torch.Tensor(np.repeat(1., 8)),},} 

In [ ]:
# iterate over all 8 experiments
for experiment in range(1, 9):
    ## Set Hyperparameters for Model & Training
    config = {
        'checkpoint_path_normal_training': f'/mnt/home/rheinrich/taaowpf/models/cnn/checkpoints_normal_training/best_resnet_model_normal_training_experiment{experiment}.ckpt',
        'checkpoint_path_adversarial_training': f'/mnt/home/rheinrich/taaowpf/models/cnn/checkpoints_adversarial_training/best_resnet_model_adversarial_training_experiment{experiment}.ckpt',
        'forecast_horizon': 8, # 8 hour ahead wind power forecast 
        'n_past_timesteps': 4, # including current time step
        'resnet_version': 34,
        'forecast_version': 'single', #'all',
        'batch_size': 64,
        'num_workers': 32,
        'learning_rate': 0.001,
        'p_adv_training': 0.,
        'eps_adv_training': 0.15,
        'step_num_adv_training': 100,
        'norm_adv_training': 'Linf',
        'step_num_noise_attack': 100,
        'lambda_weight_semi-targeted': 1000.,
    }
    
    # Initialize DataModule
    windspeed_dir = '/mnt/home/rheinrich/taaowpf/data/cnn/wind_speed_100m_germany_res10x10_012018_062021.csv'
    windpower_dir = '/mnt/home/rheinrich/taaowpf/data/cnn/windpower_germany_102018_062021.csv'
    
    datamodule = WPF_Germany_DataModule(windspeed_dir = windspeed_dir,
                                        windpower_dir = windpower_dir, 
                                        forecast_horizon = config['forecast_horizon'],  
                                        n_past_timesteps = config['n_past_timesteps'],
                                        batch_size = config['batch_size'], 
                                        num_workers = config['num_workers'],
                                        experiment = experiment,
                                       )
    
    datamodule.setup()
    
    # Initialize Forecast Model
    
    ### Load best model checkpoint
    
    #### Normal Training
    model_normal_training = WPF_ResNet.load_from_checkpoint(config['checkpoint_path_normal_training'],
                                                           resnet_version = config['resnet_version'],
                                                           forecast_version = config['forecast_version'],
                                                           forecast_horizon = config['forecast_horizon'],
                                                           n_past_timesteps = config['n_past_timesteps'],
                                                           learning_rate= config['learning_rate'],
                                                           p_adv_training = config['p_adv_training'],
                                                           eps_adv_training = config['eps_adv_training'],
                                                           step_num_adv_training = config['step_num_adv_training'],
                                                           norm_adv_training = config['norm_adv_training']
                                                          )
    
    #### Adversarial Training
    model_adversarial_training = WPF_ResNet.load_from_checkpoint(config['checkpoint_path_adversarial_training'],
                                                               resnet_version = config['resnet_version'],
                                                               forecast_version = config['forecast_version'],
                                                               forecast_horizon = config['forecast_horizon'],
                                                               n_past_timesteps = config['n_past_timesteps'],
                                                               learning_rate= config['learning_rate'],
                                                               p_adv_training = config['p_adv_training'],
                                                               eps_adv_training = config['eps_adv_training'],
                                                               step_num_adv_training = config['step_num_adv_training'],
                                                               norm_adv_training = config['norm_adv_training']
                                                              )
    
    ### Set models to evaluation mode
    model_normal_training.eval()
    model_adversarial_training.eval()
    
    # Adversarial Robustness Evaluation
    
    #### Set lower bound for perturbations, such that perturbed wind speed is never negative
    lower_bound = (0 - datamodule.mean_windspeed) / datamodule.std_windspeed
    
    ## Adversarial Robustness: NoiseAttack
    
    ### Create config for NoiseAttack
    config_noise_attack = {
        'radius': config['eps_adv_training'],
        'step_num': config['step_num_noise_attack'],
        'norm': config['norm_adv_training'],
    }
    
    ### Normal Training
    
    #### Initialize NoiseAttack
    noise_attack_normal_training = NoiseAttack(model_normal_training, MSELossBatch(), lower_bound = lower_bound)
    
    #### Evaluate Robustness to NoiseAttack
    robustness_noise_normal_training = AdversarialRobustnessEvaluator(forward_func = model_normal_training,
                                                                      dataloader = datamodule.test_dataloader(),
                                                                      attack = noise_attack_normal_training,
                                                                      attack_kwargs = config_noise_attack,
                                                                      targeted = False, 
                                                                      loss_func_prs = RMSELossBatch(),
                                                                      loss_func_drs = RMSELossBatch(),
                                                                      target_attacker = None,
                                                                      additional_inputs = False,
                                                                      requires_original_target = True,
                                                                     )
    
    results_noise_normal_training = robustness_noise_normal_training.evaluate_attack()
    robustness_noise_normal_training.summary_table()
    
    # Save all results as CSV file
    results_noise_normal_training.to_csv(f'germany_results_noise_normal-training_experiment{experiment}.csv', index = False)
    
    ### Adversarial Training
    
    #### Initialize NoiseAttack
    noise_attack_adversarial_training = NoiseAttack(model_adversarial_training, MSELossBatch(), lower_bound = lower_bound)
    
    #### Evaluate Robustness to NoiseAttack
    robustness_noise_adversarial_training = AdversarialRobustnessEvaluator(forward_func = model_adversarial_training,
                                                                           dataloader = datamodule.test_dataloader(),
                                                                           attack = noise_attack_adversarial_training,
                                                                           attack_kwargs = config_noise_attack,
                                                                           targeted = False,
                                                                           loss_func_prs = RMSELossBatch(),
                                                                           loss_func_drs = RMSELossBatch(),
                                                                           target_attacker = None,
                                                                           additional_inputs = False,
                                                                           requires_original_target = True,
                                                                          )
    
    results_noise_adversarial_training = robustness_noise_adversarial_training.evaluate_attack()
    robustness_noise_adversarial_training.summary_table()
    
    # Save all results as CSV file
    results_noise_adversarial_training.to_csv(f'germany_results_noise_adversarial-training_experiment{experiment}.csv', index = False)

    
    ## Adversarial Robustness: Untargeted PGD Attack
    
    ### Create Config for Untargeted PGD Attack
    config_pgd_untargeted = {
        'radius': config['eps_adv_training'],
        'step_size': 2 * config['eps_adv_training'] / config['step_num_adv_training'],
        'step_num': config['step_num_adv_training'],
        'norm': config['norm_adv_training'],
        'targeted': False,
    }
    
    ### Normal Training
    
    #### Initialize Untargeted PGD Attack
    pgd_untargeted_normal_training = PGD(model_normal_training, MSELossBatch(), lower_bound = lower_bound)
    
    #### Evaluate Robustness to Untargeted PGD Attack
    robustness_pgd_untargeted_normal_training = AdversarialRobustnessEvaluator(forward_func = model_normal_training,
                                                                                dataloader = datamodule.test_dataloader(),
                                                                                attack = pgd_untargeted_normal_training,
                                                                                attack_kwargs = config_pgd_untargeted,
                                                                                targeted = False,
                                                                                loss_func_prs = RMSELossBatch(),
                                                                                loss_func_drs = RMSELossBatch(),
                                                                                target_attacker = None,
                                                                                additional_inputs = False,
                                                                                requires_original_target = True,
                                                                               )
    
    results_pgd_untargeted_normal_training = robustness_pgd_untargeted_normal_training.evaluate_attack()
    robustness_pgd_untargeted_normal_training.summary_table()
    
    # Save all results as CSV file
    results_pgd_untargeted_normal_training.to_csv(f'germany_results_pgd_untargeted_normal-training_experiment{experiment}.csv', index = False)
    
    ### Adversarial Training
    
    #### Initialize Untargeted PGD Attack
    pgd_untargeted_adversarial_training = PGD(model_adversarial_training, MSELossBatch(), lower_bound = lower_bound)
    
    #### Evaluate Robustness to Untargeted PGD Attack
    robustness_pgd_untargeted_adversarial_training = AdversarialRobustnessEvaluator(forward_func = model_adversarial_training,
                                                                                    dataloader = datamodule.test_dataloader(),
                                                                                    attack = pgd_untargeted_adversarial_training,
                                                                                    attack_kwargs = config_pgd_untargeted,
                                                                                    targeted = False,
                                                                                    loss_func_prs = RMSELossBatch(),
                                                                                    loss_func_drs = RMSELossBatch(),
                                                                                    target_attacker = None,
                                                                                    additional_inputs = False,
                                                                                    requires_original_target = True,
                                                                                   )
    
    results_pgd_untargeted_adversarial_training = robustness_pgd_untargeted_adversarial_training.evaluate_attack()
    robustness_pgd_untargeted_adversarial_training.summary_table()
    
    # Save all results as CSV file
    results_pgd_untargeted_adversarial_training.to_csv(f'germany_results_pgd_untargeted_adversarial-training_experiment{experiment}.csv', index = False)    
    
    
    # iterate over all of the attacker's targets
    for target in target_attacker_dict.keys():
        target_attacker = torch.tensor(target_attacker_dict[target])
        
        ## Adversarial Robustness: Targeted PGD Attack
        
        ### Create Config for Targeted PGD Attack
        config_pgd_targeted = {
            'radius': config['eps_adv_training'],
            'step_size': 2 * config['eps_adv_training'] / config['step_num_adv_training'],
            'step_num': config['step_num_adv_training'],
            'norm': config['norm_adv_training'],
            'targeted': True,
        }
        
        ### Normal Training
        
        #### Initialize Targeted PGD Attack
        pgd_targeted_normal_training = PGD(model_normal_training, MSELossBatch(), lower_bound = lower_bound)
        
        #### Evaluate Robustness to Targeted PGD Attack
        robustness_pgd_targeted_normal_training = AdversarialRobustnessEvaluator(forward_func = model_normal_training,
                                                                            dataloader = datamodule.test_dataloader(),
                                                                            attack = pgd_targeted_normal_training,
                                                                            attack_kwargs = config_pgd_targeted,
                                                                            targeted = True,
                                                                            loss_func_prs = RMSELossBatch(),
                                                                            loss_func_drs = RMSELossBatch(),
                                                                            target_attacker = target_attacker,
                                                                            additional_inputs = False,
                                                                            requires_original_target = False,
                                                                           )
        
        results_pgd_targeted_normal_training = robustness_pgd_targeted_normal_training.evaluate_attack()
        robustness_pgd_targeted_normal_training.summary_table()
        
        # Save all results as CSV file
        results_pgd_targeted_normal_training.to_csv(f'germany_results_pgd_targeted_normal-training_target-{target}_experiment{experiment}.csv', index = False)
    
        ### Adversarial Training
        
        #### Initialize Targeted PGD Attack
        pgd_targeted_adversarial_training = PGD(model_adversarial_training, MSELossBatch(), lower_bound = lower_bound)
        
        #### Evaluate Robustness to Targeted PGD Attack
        robustness_pgd_targeted_adversarial_training = AdversarialRobustnessEvaluator(forward_func = model_adversarial_training,
                                                                                dataloader = datamodule.test_dataloader(),
                                                                                attack = pgd_targeted_adversarial_training,
                                                                                attack_kwargs = config_pgd_targeted,
                                                                                targeted = True,
                                                                                loss_func_prs = RMSELossBatch(),
                                                                                loss_func_drs = RMSELossBatch(),
                                                                                target_attacker = target_attacker,
                                                                                additional_inputs = False,
                                                                                requires_original_target = False,
                                                                               )
        
        results_pgd_targeted_adversarial_training = robustness_pgd_targeted_adversarial_training.evaluate_attack()
        robustness_pgd_targeted_adversarial_training.summary_table()
        
        # Save all results as CSV file
        results_pgd_targeted_adversarial_training.to_csv(f'germany_results_pgd_targeted-adversarial_training_target-{target}_experiment{experiment}.csv', index = False)
        
        
    # iterate over all of the attacker's semi-targets
    for semi_target in semi_target_attacker_dict.keys():
        semi_target_attacker = semi_target_attacker_dict[semi_target]
        
        ## Adversarial Robustness: Semi-Targeted PGD Attack
        
        ### Create Config for Semi-Targeted PGD Attack
        config_pgd_semitargeted = {
            'radius': config['eps_adv_training'],
            'step_size': 2 * config['eps_adv_training'] / config['step_num_adv_training'],
            'step_num': config['step_num_adv_training'],
            'norm': config['norm_adv_training'],
            'targeted': False,
        }
        
        mse_loss_semitargeted = MSELossBatch_SemiTargeted(lower_bound = semi_target_attacker['lower_bound'],
                                                          upper_bound = semi_target_attacker['upper_bound'],
                                                          lambda_weight = config['lambda_weight_semi-targeted'],
                                                         )
        
        rmse_area = RMSELossBatch_Area(lower_bound = semi_target_attacker['lower_bound'],
                                       upper_bound = semi_target_attacker['upper_bound'],
                                      )
        
        ### Normal Training
        
        #### Initialize Semi-Targeted PGD Attack
        pgd_semitargeted_normal_training = PGD(model_normal_training, mse_loss_semitargeted, lower_bound = lower_bound)
        
        #### Evaluate Robustness to Semi-Targeted PGD Attack
        robustness_pgd_semitargeted_normal_training = AdversarialRobustnessEvaluator(forward_func = model_normal_training,
                                                                                    dataloader = datamodule.test_dataloader(),
                                                                                    attack = pgd_semitargeted_normal_training,
                                                                                    attack_kwargs = config_pgd_semitargeted,
                                                                                    targeted = False,
                                                                                    loss_func_prs = RMSELossBatch(),
                                                                                    loss_func_drs = rmse_area,
                                                                                    target_attacker = torch.tensor(np.repeat(np.nan, 8)),
                                                                                    additional_inputs = False,
                                                                                    requires_original_target = True,
                                                                                   )
        
        results_pgd_semitargeted_normal_training = robustness_pgd_semitargeted_normal_training.evaluate_attack()
        robustness_pgd_semitargeted_normal_training.summary_table()
        
        # Save all results as CSV file
        results_pgd_semitargeted_normal_training.to_csv(f'germany_results_pgd_semi-targeted_normal-training_target-{semi_target}_experiment{experiment}.csv', index = False)
        
        
        ### Adversarial Training
        
        #### Initialize Semi-Targeted PGD Attack
        pgd_semitargeted_adversarial_training = PGD(model_adversarial_training, mse_loss_semitargeted, lower_bound = lower_bound)
        
        #### Evaluate Robustness to Semi-Targeted PGD Attack
        robustness_pgd_semitargeted_adversarial_training = AdversarialRobustnessEvaluator(forward_func = model_adversarial_training,
                                                                                        dataloader = datamodule.test_dataloader(),
                                                                                        attack = pgd_semitargeted_adversarial_training,
                                                                                        attack_kwargs = config_pgd_semitargeted,
                                                                                        targeted = False,
                                                                                        loss_func_prs = RMSELossBatch(),
                                                                                        loss_func_drs = rmse_area,
                                                                                        target_attacker = torch.tensor(np.repeat(np.nan, 8)),
                                                                                        additional_inputs = False,
                                                                                        requires_original_target = True,
                                                                                       )
        
        results_pgd_semitargeted_adversarial_training = robustness_pgd_semitargeted_adversarial_training.evaluate_attack()
        robustness_pgd_semitargeted_adversarial_training.summary_table()
        
        # Save all results as CSV file
        results_pgd_semitargeted_adversarial_training.to_csv(f'germany_results_pgd_semi-targeted_adversarial-training_target-{semi_target}_experiment{experiment}.csv', index = False)
